In [ ]:
 import numpy as np
 import pandas as pd
 import tensorflow as tf
 import matplotlib.pyplot as plt
 from tensorflow import keras
 %matplotlib inline
 from sklearn.model_selection import train_test_split as tts
 from sklearn.preprocessing import StandardScaler
 import math

In [ ]:
df=pd.read_csv('/content/insurance_data.csv')
df.head()

,age,affordibility,bought_insurance
0,22,1,0
1,25,0,0
2,47,1,1
3,52,0,0
4,46,1,1


In [ ]:
df["age"]=df["age"]/100

In [ ]:
x_train,x_test,y_train,y_test=tts(df[["age","affordibility"]],df['bought_insurance'],test_size=0.25)

In [ ]:
model=keras.Sequential([
    keras.layers.Dense(1,input_shape=(2,),activation='sigmoid',kernel_initializer="ones",bias_initializer="zeros")
])
model.compile(optimizer="adam",
              loss="binary_crossentropy",
              metrics=['accuracy'])
model.fit(x_train,y_train,epochs=500)

Epoch 1/500


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 973ms/step - accuracy: 0.4286 - loss: 0.7859
Epoch 2/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step - accuracy: 0.4286 - loss: 0.7854
Epoch 3/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step - accuracy: 0.4286 - loss: 0.7848
Epoch 4/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step - accuracy: 0.4286 - loss: 0.7843
Epoch 5/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 64ms/step - accuracy: 0.4286 - loss: 0.7838
Epoch 6/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step - accuracy: 0.4286 - loss: 0.7833
Epoch 7/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 67ms/step - accuracy: 0.4286 - loss: 0.7827
Epoch 8/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 129ms/step - accuracy: 0.4286 - loss: 0.7822
Epoch 9/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step - accuracy: 0.4286 - loss: 0.7817
Epoch 10/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 62ms/step - accuracy: 0.4286 - loss: 0.7812
Epoch 11/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 58ms/step - accuracy: 0.4286 - loss: 0.7806
Epoch 12/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step - accuracy: 0.4286 - loss: 0.7801

In [ ]:
y_pre=model.evaluate(x_test,y_test)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 128ms/step - accuracy: 0.7143 - loss: 0.5700


In [ ]:
model.predict(x_test)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step


array([[0.6411818 ],
       [0.59008604],
       [0.6396733 ],
       [0.6259754 ],
       [0.43704882],
       [0.49403274],
       [0.6320898 ]], dtype=float32)

In [ ]:
coef,intercept=model.get_weights()
coef,intercept

(array([[0.655079  ],
        [0.63712144]], dtype=float32),
 array([-0.41691762], dtype=float32))

In [ ]:
def sigmoid(x):
  return 1/(1+math.exp(-x))
sigmoid(18)

0.9999999847700205

In [ ]:
def predict(age,affordability):
  r=coef[0]*age+coef[1]*affordability+intercept
  return sigmoid(r)
predict(0.79,0)

<ipython-input-16-f561d334420a>:2: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  return 1/(1+math.exp(-x))


0.5251275187757715

In [ ]:
x_test

,age,affordibility
15,0.55,1
0,0.22,1
25,0.54,1
23,0.45,1
1,0.25,0
7,0.60,0
14,0.49,1


In [ ]:
def log_loss(y_true,y_predicted):
  epsilon=1e-15
  y_pre_new = [max(i,epsilon)for i in y_predicted]
  y_pre_new = [min(i,epsilon)for i in y_pre_new]
  y_pre_new = np.array(y_pre_new)
  return -np.mean(y_true*np.log(y_pre_new)+(1-y_true)*np.log(1-y_pre_new))

In [ ]:
def sigmoid_np(x):
  return 1/(1+np.exp(-x))
sigmoid_np(np.array([12,0,5]))

array([0.99999386, 0.5       , 0.99330715])

In [ ]:
def gradient_descent(age,affordability,y_true,epochs):
  w1=w2=1
  bias=0
  rate=.5
  n=len(age)
  for i in range(epochs):
    r=w1*age+w2*affordability+bias
    y_pre=sigmoid_np(r)
    loss=log_loss(y_true,y_pre)
    w1d=(1/n)*np.dot(np.transpose(age),(y_pre-y_true ))
    w2d=(1/n)*np.dot(np.transpose(affordability),(y_pre-y_true ))
    bias_d=np.mean(y_pre-y_true)
    w1=w1-rate*w1d
    w2=w2-rate*w2d
    bias=bias-rate*bias_d
    print(f"epochs:{i}, w1:{w1},w2:{w2},bias:{bias},loss:{loss}")
  return w1,w2,bias

In [ ]:
gradient_descent(x_train["age"],x_train["affordibility"],y_train,1000)

epochs:0, w1:0.9619643015914773,w2:0.9229498001234338,bias:-0.1502536000206826,loss:14.802332740676006
epochs:1, w1:0.9316273359834405,w2:0.8595809444777176,bias:-0.28017705670806836,loss:14.802332740676006
epochs:2, w1:0.9084771582957146,w2:0.8092429404453374,bias:-0.39127585740588255,loss:14.802332740676006
epochs:3, w1:0.8917434341449169,w2:0.770682095242862,bias:-0.4856679380881361,loss:14.802332740676006
epochs:4, w1:0.8805411976830161,w2:0.7423363970114413,bias:-0.5657144633743016,loss:14.802332740676006
epochs:5, w1:0.8739856901521226,w2:0.722582885494632,bias:-0.6337353476949681,loss:14.802332740676006
epochs:6, w1:0.8712631535235462,w2:0.709896981468788,bias:-0.691840356032964,loss:14.802332740676006
epochs:7, w1:0.8716634377411348,w2:0.7029315985589859,bias:-0.7418563482954927,loss:14.802332740676006
epochs:8, w1:0.8745872605837801,w2:0.7005414034955805,bias:-0.7853164477880975,loss:14.802332740676006
epochs:9, w1:0.8795396529258207,w2:0.7017762013569885,bias:-0.8234818437204

(7.757545786802065, 1.8447951083890595, -4.671081342471358)

In [ ]:
coef,intercept

(array([[0.655079  ],
        [0.63712144]], dtype=float32),
 array([-0.41691762], dtype=float32))